
#Proving the correctness of the $(K3,K3)$-sender

In this notebook, we will
look at SMT solvers and see how they can be used to solve otherwise difficult problems. Particularly, we will prove the correctness of the $(K_3,K_3)$-sender, a gadget we used to prove the hardness of $(K_3,K_3)$-nonarrowing earlier this semester.

Recall that a **$(K_3,K_3)$-sender** is a graph $G$ such that

1.   $G$ is $(K_3,K_3)$-good,
2.   There exists two edges, called **sender edges**, $e_1, e_2$ in $E(G)$ such that $e_1$ is red iff $e_2$ is red in all good colorings of $G$.


We observed that the graph obtained as follows is a $(K_3, K_3)$-sender:
"Take two copies of K5 and identify a triangle in each. So, the gadget has 7 vertices. The two edges not connected to the shared triangle have the same color in any $(K_3, K_3)$-good coloring." From this point on, we will refer to the graph described as $G$.

Below, we will first construct $G$ using a library called *NetworkX*, then encode the nonarrowing problem on $G$ using a boolean formula. Finally, we will
use an SMT solver, *Z3*, to verify that sender edges always have the same color is good colorings of $G$.


**Instructions:**
1. To get started, click on File on the top left and click "Save a copy in Drive."
This will give you an editable version of this document that you can use.
2. If you press `CMD`+`Enter` it runs the cell, and if you press `Shift`+`Enter` it runs the cell and goes to the next one.
3. Make sure you run all cells as you go through the notebook; some cells will not work properly unless the previous one
has been run too.
4. If you disconnect or are inactive for some time you should run all of the cells again.

## 0. Preliminaries (you should run this cell but there is no need to read it)

In [ ]:
!pip install z3-solver
!pip install git+https://github.com/crrivero/FormalMethodsTasting.git#subdirectory=core
from z3 import *
from tofmcore import showSolver, drawJ4, drawG, drawGandColoring, list_all_solutions
import networkx as nx
import itertools
from IPython.display import clear_output
clear_output()

## Encoding constraints in Z3

The goal of this notebook is to teach you about formal methods;
particularly, how you can use existing formal verification tools
(in this case, Z3) to analyze and solve your own problems.
Before we get started, let's look at some basic things we can do with Z3.

### Boolean

Suppose you have the following three boolean constraints and you want to check if there's a solution (an assignment of the variables) that satisfies all of them:

$$ x_1 \lor x_2 \lor x_3 $$

$$ \neg x_1 \implies \neg x_2$$

$$  x_1 \land x_3  $$

Let's see how we can do this using Z3.

In [ ]:
s = Solver() # initialize Z3 solver

# initialize variables

x1 = Bool('x_1') # declaring that x_1 is a boolean variable in Z3 which will be referred to as x1 in Python
x2 = Bool('x_2')
x3 = Bool('x_3')

# Note: we can also initialize multiple variables like so: x1, x2, x3 = Bools('x_1 x_2 x_3')

# we use s.add(.) to add a constraint to our solver s
# constraints can be made using many different operations such as Or, And, Not,
# equality, etc.

# here's how we would add the constraints above to our solver:

s.add( Or( x1, x2, x3 ) ) # add the first constraint
s.add( Implies( Not(x1), Not(x2) ) ) # add the second constraint
s.add( And( x1, x3 ) ) # add the third constraint

In [ ]:
# to view the constraints in our solver, we can use the following:
print( s )
# this prints the constraints as they appear in Z3 using Z3's notation

For better readability, this notebook also has a custom print function to view our constraints in LaTeX format, like so:

In [ ]:
showSolver( s )

In [ ]:
# we can use s.check() to run the solver and check whether its satisfiable:
print ( s.check() )

 "sat" means our system of constraints is satisfiable

In [ ]:
# after using s.check(),  we can use s.model() to output a solution if one exsits
solution = s.model()
print( solution )

Let's modify our system of constraint a bit and see if it's still satisfiable. Suppose we want to check if there's a solution where $x_1 = \neg x_3$. Let's see how we would do this with Z3.

In [ ]:
s.add( x1 == Not(x3) )
showSolver( s )

In [ ]:
print( s.check() ) # check if solution exists with new constraint

"unsat" means the system is not satisfiable, i.e., there is no assignment on the variables $x_1$, $x_2$, and $x_3$ that satisfies all the constraints we gave to the solver. **Note that if we were to run s.model() now we would get an error.**

## Constructing $G$

In [ ]:
# Our first step is to construct $G$ as a NetworkX graph.

# This simplest (albiet, most verbose) way to do this is by using a list of edges
# For example, if you wanted to construct a J4 (a K4 missing an edge)
# you can do it like so:
J4 = nx.Graph( [ (0,1), (0,3), (1,2), (1,3), (2,3) ] )

# To see what this looks like, we've defined a special function above that uses
# NetworkX's drawing capabilities
drawJ4(J4)

In [ ]:
# Now it's your turn! Complete the following definition of G to construct
# the sender described in the introduction. When constructing the graph, use the
# vertex labels 0, 1, 2, 3, 4, 5, and 6, and make sure that (0,1) and (5,6)
# are your sender edges

# Fun fact: instead of listing the edges, you can also construct the graph
# using NetworkX's built-in functions, nx.complete_graph(.) and nx.compose(.)
# but you should do whatever feels easier.

G = nx.Graph([(0,1),(5,6)]) # replace this line

# Once you've constructed the graph, you can run the next cell function to draw
# it and verify whether you've done it correctly. The sender edges will be bolded

In [ ]:
drawG(G)

## Encoding $(K_3, K_3)$-nonarrowing as boolean formulas

Recall that a graph is $(K_3, K_3)$-good coloring of $G$ is one where there are no red triangles and no blue triangles, when the edges of $G$ are colored red and blue. We will now see how to encode this as a boolean formula.

Suppose we represent each edge $e \in E(G)$ as a variable $x_e$. Note that $e$ has two state's w.r.t. what color it can be: red or blue. Similarly, $x_e$ can either be true or false. Thus, we'll say that $x_e$ is true iff $e$ is blue (the rhyme should make this correspondence easy to remember 😊).

Thus, if we can construct or-clause(s) for each triangle that don't allow all of its edges to be the same color, the conjunction of all these clauses must give us a formula $\phi$ such that each satisfying assignment of $\phi$ will correspond to a good coloring of $G$.

Below, we will see how to represent boolean formulas in Z3 and find their satisfying assignments.

### Finding Triangles in Graphs

In [ ]:
# First, we'll need a function that checks whether three given edges
# form a triangle. This is easily done by checking the indices of the edges'
# vertices, or counting the number of vertices and edges on the graph induced
# by the given edges. Or, if you want to be clever, you can use some of
# NetworkX's built-in isomorphism checking functions

# You can assume that e1 e2 and e3 are 2-tuples of integers
def isK3( e1, e2, e3 ):
  return False

In [ ]:
# As a test, check that the following function returns True

isK3( (5,6), (5,231), (6,231) )

In [ ]:
# the following returns False

isK3( (5,6), (5,231), (6,232) )

In [ ]:
# the following returns False

isK3( (5,6), (5,231), (5,4) )

### Constructing $G$'s formula

Complete the code below to construct our desired formula.

In [ ]:
edges = G.edges() # get list of edges

# First, we map each edge to a unique boolean variable

eMap = {} # a dictionary mapping each edge in E(G) to a variable
eMapInv = {} # the inverse of the dictionary above

idx = 1
for e in edges:
    b_name = 'x'+str(idx)
    b = Bool(b_name)
    eMap[e] = b
    eMapInv[b_name] = e
    idx += 1
s = Solver() # initialize solver

for e1, e2, e3 in itertools.combinations( edges, 3 ): # iterate over all 3-tuples of edges
    # if e1 e2 and e3 form a triangle
    if ( isK3(e1, e2, e3) ):
        # add clauses to solver

        e1Var = eMap[e1]
        e2Var = eMap[e2]
        e3Var = eMap[e3]

        # ADD YOUR CODE HERE

        s.add( Or(e1Var) ) # replace this line
        s.add( Or(Not(e2Var)) ) # replace this line

# we have defined a function that, given a Z3 solver, obtains the list of all
# possible solutions
good_colorings = list_all_solutions( s, list(eMap.values()) )

In [ ]:
# Assuming everything is done correctly, we should have 24 solutions,
# each of which corresponds to a good coloring of G
print(len(good_colorings))

In [ ]:
# To view these solutions, run the code in this cell with an index between 0
# and 23 to view one of the good colorings of G
drawGandColoring(G, eMapInv, good_colorings, index=0)

Now, we have two ways to verify that $(0,1)$ and $(5,6)$ are always the same color

1.   Enumerate over all colorings and check that the edges are always the same color
2.   Add extra clause(s) to $\phi$ that forces $(0,1)$ and $(5,6)$ to be the same color and verify that there are no solutions to this new formula

We will opt for the latter, simpler solution, given our newfound SMT-solving prowess.

In [ ]:
# We must first reinitialize our solver, since enumerating over all solutions
# makes the solver unsatisfiable

s = Solver() # initialize solver

for e1, e2, e3 in itertools.combinations( edges, 3 ): # iterate over all 3-tuples of edges
    # if e1 e2 and e3 form a triangle
    if ( isK3(e1, e2, e3) ):
        # add clauses to solver

        e1Var = eMap[e1]
        e2Var = eMap[e2]
        e3Var = eMap[e3]

        # ADD YOUR CODE HERE

        s.add( Or(e1Var) ) # replace this line, same as before
        s.add( Or(Not(e2Var)) ) # replace this line, same as before


# Add the new clause(s) to phi below

e1Var = eMap[ (0,1) ]
e2Var = eMap[ (5,6) ]

# ADD YOUR CODE HERE

s.add( Or(Not(e2Var)) ) # replace this line

# The following should return unsat
s.check()


### Congratulations! You just proved that $G$ is a valid $(K_3, K_3)$-sender!


####If you'd like to continue your Z3 journey, you can start with this guide to learn more:
https://ericpony.github.io/z3py-tutorial/guide-examples.htm